In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk("./"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Frame the problem
Using the customer description, Define the problem your trying to solve in your own words (remember this is not technial but must be specific so the customer understands the project

In [2]:
#Scrape housing Data from San Francisco Area to accurately predict housing prices

# 2. Get the Data 
Define how you recieved the data (provided, gathered..)

In [6]:
from pathlib import Path
from datetime import datetime
import pandas as pd
from homeharvest import scrape_property

TS = datetime.now().strftime("%Y%m%d_%H%M%S")
OUT = Path(f"SF_HomeHarvest_{TS}.csv")


raw = scrape_property(
    location="San Francisco, CA",
    listing_type="sold",         
    past_days=200,
    extra_property_data=True
)

print(f"Fetched {len(raw):,} rows before filtering")

df = raw.copy()
city_cols = [c for c in df.columns if c.lower() in {"city"}]
if city_cols:
    df = df[df[city_cols[0]].str.fullmatch("San Francisco", na=False)]
else:
    addr_col = next((c for c in df.columns if "address" in c.lower() or "permalink" in c.lower()), None)
    if addr_col:
        df = df[df[addr_col].str.contains(", San Francisco, CA", na=False)]


df = df.drop_duplicates(subset=[c for c in df.columns if "id" in c.lower()] or None)
df.to_csv(OUT, index=False)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# 3. Explore the Data
Gain insights into the data you have from step 2, making sure to identify any bias

In [7]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split

SRC = "SF_HomeHarvest_20250910_235144.csv"  
df = pd.read_csv(SRC)

keep_like = {
    "address": [c for c in df.columns if "address" in c.lower()] or [None],
    "price":   [c for c in df.columns if "price" in c.lower() and "list" not in c.lower()],
    "beds":    [c for c in df.columns if "bed" in c.lower()],
    "baths":   [c for c in df.columns if "bath" in c.lower()],
    "area":    [c for c in df.columns if "sq" in c.lower() or "area" in c.lower()],
    "type":    [c for c in df.columns if "type" in c.lower() or "style" in c.lower()],
}
addr = keep_like["address"][0]
price = keep_like["price"][0]
beds = keep_like["beds"][0]
baths = keep_like["baths"][0]
area = keep_like["area"][0]
ptype = keep_like["type"][0]

core = df[[addr, price, beds, baths, area, ptype]].rename(columns={
    addr: "Address",
    price: "SoldPrice",
    beds: "Bedrooms",
    baths: "Bathrooms",
    area: "AreaSqFt",
    ptype: "PropertyType",
})

def sf_neighborhood(addr: str) -> str:
    if not isinstance(addr, str):
        return "San Francisco"
    m = re.search(r",\s*([^,]+),\s*San Francisco,\s*CA", addr)
    return m.group(1).strip() if m else "San Francisco"

core["Neighborhood"] = core["Address"].apply(sf_neighborhood)

core.to_csv("sf_core.csv", index=False)
train, test = train_test_split(core, test_size=0.2, random_state=1337)
train.to_csv("sf_train.csv", index=False)
test.to_csv("sf_test.csv", index=False)
print(train.shape, test.shape)

(2844, 7) (711, 7)


# 4.Prepare the Data


Apply any data transformations and explain what and why


In [11]:
import re, glob
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

candidates = sorted(glob.glob("SF_HomeHarvest_20250910_235144.csv") + glob.glob("sf_core.csv"))
SRC = candidates[-1]
df = pd.read_csv(SRC)

def pick(*keys):
    keys = [k.lower() for k in keys]
    # exact-ish preference pass
    for c in df.columns:
        lc = c.lower()
        if any(k == lc for k in keys):
            return c
    # substring fallback pass
    for c in df.columns:
        lc = c.lower()
        if any(k in lc for k in keys):
            return c
    return None

addr  = pick("address", "full_address", "permalink", "property_url")
price = pick("sold price", "closing price", "sold_price", "price")
beds  = pick("bedrooms", "beds", "bed")
baths = pick("bathrooms", "baths", "bath")
area  = pick("area (sqft)", "sqft", "area", "living area")
ptype = pick("property type", "style", "type")

found = {"addr":addr,"price":price,"beds":beds,"baths":baths,"area":area,"ptype":ptype}
print("Picked columns:", found)

if any(v is None for v in found.values()):
    missing = [k for k, v in found.items() if v is None]
    raise ValueError(f"Could not find required columns for: {missing}. "
                     f"Available cols: {list(df.columns)}")

core = df[[addr, price, beds, baths, area, ptype]].rename(columns={
    addr:"Address", price:"SoldPrice", beds:"Bedrooms",
    baths:"Bathrooms", area:"AreaSqFt", ptype:"PropertyType"
})

core = core[core["Address"].astype(str).str.contains(r",\s*San Francisco,\s*CA", na=False)]
if core.empty:
    raise ValueError("SF filter removed all rows. Check that addresses contain ', San Francisco, CA'.")

core["Neighborhood"] = core["Address"].str.extract(r",\s*([^,]+),\s*San Francisco,\s*CA")
core["Neighborhood"] = core["Neighborhood"].fillna("San Francisco")

for col in ["SoldPrice","Bedrooms","Bathrooms","AreaSqFt"]:
    core[col] = pd.to_numeric(core[col], errors="coerce")
    core[col] = core[col].fillna(core[col].median())

if core["PropertyType"].isna().all():
    core["PropertyType"] = "Unknown"
else:
    core["PropertyType"] = core["PropertyType"].fillna(core["PropertyType"].mode(dropna=True).iloc[0])

core["PricePerSqFt"] = np.where(core["AreaSqFt"] > 0,
                                core["SoldPrice"] / core["AreaSqFt"],
                                core["SoldPrice"])

final_cols = ["Bedrooms","Bathrooms","AreaSqFt","PricePerSqFt",
              "PropertyType","Neighborhood","SoldPrice","Address"]
core = core[final_cols].drop_duplicates().reset_index(drop=True)
core.to_csv("sf_clean.csv", index=False)

train, test = train_test_split(core, test_size=0.2, random_state=1337)
train.to_csv("sf_train.csv", index=False)
test.to_csv("sf_test.csv", index=False)

print(f"Clean rows: {len(core)} | Train: {len(train)} | Test: {len(test)}")


Picked columns: {'addr': 'Address', 'price': 'SoldPrice', 'beds': 'Bedrooms', 'baths': 'Bathrooms', 'area': 'AreaSqFt', 'ptype': 'PropertyType'}
Clean rows: 3554 | Train: 2843 | Test: 711


# 5. Model the data
Using selected ML models, experment with your choices and describe your findings. Finish by selecting a Model to continue with


In [12]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor

df = pd.read_csv("sf_core.csv").dropna(subset=["SoldPrice"])
y = df["SoldPrice"].astype(float)
X = df.drop(columns=["SoldPrice", "Address"])

num_cols = ["Bedrooms", "Bathrooms", "AreaSqFt"]
cat_cols = ["PropertyType", "Neighborhood"]

pre = ColumnTransformer(
    [
        ("num", Pipeline([("imp", SimpleImputer(strategy="median")),
                          ("sc", StandardScaler())]), num_cols),
        ("cat", Pipeline([("imp", SimpleImputer(strategy="most_frequent")),
                          ("oh", OneHotEncoder(handle_unknown="ignore"))]), cat_cols),
    ]
)

models = {
    "HGBR": HistGradientBoostingRegressor(max_depth=6, learning_rate=0.08, max_iter=600, random_state=1337),
    "RF":   RandomForestRegressor(n_estimators=350, max_depth=8, min_samples_leaf=4, random_state=1337, n_jobs=-1),
}

Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=1337)

for name, est in models.items():
    pipe = Pipeline([("pre", pre), ("est", est)])
    pipe.fit(Xtr, ytr)
    pred = pipe.predict(Xte)
    mae = mean_absolute_error(yte, pred)
    r2  = r2_score(yte, pred)
    print(f"{name}: MAE=${mae:,.0f}  R²={r2:0.3f}")


HGBR: MAE=$755,728  R²=0.171
RF: MAE=$693,936  R²=0.318


# 6. Fine Tune the Model

With the select model descibe the steps taken to acheve the best rusults possiable 


# 7. Present
In a customer faceing Document provide summery of finding and detail approach taken


# 8. Launch the Model System
Define your production run code, This should be self susficent and require only your model pramaters 


In [ ]:
def infrence(prams):
    results = m.run(prams)
    return results 